In [1]:
import re
import os
import sys
import csv
import numpy as np 
import pandas as pd
import seaborn as sns
from matplotlib_venn import venn2
from prettytable import PrettyTable
from matplotlib import pyplot as plt
from Bio import SeqIO, pairwise2, AlignIO
from collections import Counter, namedtuple
from Bio.Align import AlignInfo, MultipleSeqAlignment
import importlib
from Sequence import Sequence
from Util.SeqUtil import seqInfo, parseFasta
from Evaluation.DfamEvaluation import DfamEvaluation
from DataStructure import PositionInfo, refSeqSimilarityInfo
from SharedInfo import currDatasetName, cutterA, cutterB, colorA, colorB
from Util.AnalysisUtil import listToSortedCounter, getStatisticData, mostCommonTable
from Util.PlotUtil import basicPlot, twoLabelBasicPlot, lengthScatterDistributionPlot
from MultipleCutter import MultipleCutter
# importlib.reload(sys.modules['Evaluation.DfamEvaluation'])

<module 'Evaluation.DfamEvaluation' from '/Users/apple/Desktop/BioProject/GenomeAnalysis/src/Evaluation/DfamEvaluation.py'>

In [2]:
# ROO_LTR_df = pd.read_csv("./Evaluation/Source/chrX_ROO_LTR_repeatSeq.csv")
# ROO_counter = Counter(list(ROO_LTR_df['length']))
# ROO_counter.most_common(len(ROO_counter))

In [3]:
currDatasetName = "chrX_dm6"

In [4]:
seqA = Sequence(cutterA)
parseFastaA = seqA.parseFasta()
fragmentLenListA, fragmentSeqListA = seqA.parseSeqByCutter()
repeatInfoListA = seqA.findRepeatSeqs(lengthLimit=False)
filterRepeatInfoA = seqA.filterRepeatInfo()
repeatPositionListA = seqA.getRepeatPositionList(filter=False)

...start parsing dm6/chrX_sequence.fasta fasta file ...
...cost0.1544339656829834 sec to parse fasta file ...
...start parse seq by cutter: GATC
...cost 0.2915968894958496 sec to cut sequence
... start finding repeat seq ...
...cost0.06168079376220703 sec to finding repeat seq  ...


In [5]:
seqB = Sequence(cutterB)
parseFastaB = seqB.parseFasta()
fragmentLenListB, fragmentSeqListB = seqB.parseSeqByCutter()
repeatInfoListB = seqB.findRepeatSeqs(lengthLimit=False)
filterRepeatInfoB = seqB.filterRepeatInfo()
repeatPositionListB = seqB.getRepeatPositionList(filter=False)

...start parsing dm6/chrX_sequence.fasta fasta file ...
...cost0.14101600646972656 sec to parse fasta file ...
...start parse seq by cutter: AAGCTT
...cost 0.31345105171203613 sec to cut sequence
... start finding repeat seq ...
...cost0.005834102630615234 sec to finding repeat seq  ...


In [6]:
seqInfo(currDatasetName, parseFastaA)

chrX_dm6 dataset
 number of sequence:1
 total length:23542271



In [36]:
repeatPositionList = repeatPositionListA + repeatPositionListB

In [37]:
print(f'Check cutter A, B: \n {len(repeatPositionList)} = {len(repeatPositionListA)} + {len(repeatPositionListB)}')

Check cutter A, B: 
 76663 = 73708 + 2955


In [9]:
repeatPositionList[0]

PositionInfo(startIdx=-4, endIdx=432)

In [10]:
# df = pd.DataFrame(columns=['startIdx', 'endIdx', 'length'])
# for i in repeatPositionList:
#     df = df.append({'startIdx': i.startIdx, 'endIdx': i.endIdx, 'length': i.endIdx-i.startIdx}, ignore_index=True)
# df.to_csv(f'../outputFile/txtFile/NonFilter_Position.csv')

In [60]:
# repeat position
multipleCutter = MultipleCutter(chrLength=len(parseFastaA[0]),repeatPositionList = repeatPositionList )
seqStateList = multipleCutter.seqStateGenerator()
unMatchState, unionState, intersectionState = multipleCutter.getSeqStateInfo()
stateName="union"
matchStateIdxList = multipleCutter.getSpecificStateIdxList(stateName)
matchStatePositionList = multipleCutter.getSpecificStatePositionList()

chr: 23542271
unMatch: 1411628, union:21759007, intersection:5797805


In [62]:
seqStateList[len(seqStateList)-10:]

[2, 2, 2, 2, 3, 3, 4, 4, 4, 4]

In [31]:
# Output MultipleCutter State Position
textfile = open(f"../outputFile/txtFile/{stateName}StatePosition.txt", "w")
for element in matchStatePositionList:
    textfile.write(str(element) + "\n")
textfile.close()

In [12]:
# df = pd.DataFrame(columns=['startIdx', 'endIdx', 'length'])
# for i in matchStatePositionList:
#     df = df.append({'startIdx': i.startIdx, 'endIdx': i.endIdx, 'length': i.endIdx-i.startIdx}, ignore_index=True)

In [13]:
# df['length'] = df['length'].astype('int32')
# df['length'].describe()

In [14]:
# df.to_csv(f'../outputFile/txtFile/{stateName}AndNonFilter_Position.csv')

In [15]:
# repeatPositionList = matchStatePositionList

In [38]:
dfam = DfamEvaluation(matchStatePositionList, hitFileName='chrX_LTR_dm6_dfam.nrph.hits')

In [39]:
repeatPositionLookupDic = dfam.positionBucketClassifier()
dfamPositionList = dfam.getDfamPositionList()

In [40]:
# from Dfam , check repeat
DRrepeatMatchList, DRmatchedFamilyAccList, DRmatchedFamilyNameList = dfam.checkDfamMatchWithRepeat()

In [41]:
textfile = open(f"../outputFile/txtFile/DRrepeatMatchList_{stateName}StatePosition.txt", "w")
for element in DRrepeatMatchList:
    textfile.write(str(element) + "\n")
textfile.close()

In [42]:
# from repeat , check Dfam
# RDrepeatMatchList, RDmatchedFamilyAccList, RDmatchedFamilyNameList = dfam.checkRepeatMatchWithDfam()

In [43]:
# dfam.familyMatchRatio(DRmatchedFamilyAccList)
dfam.matchRatio(DRrepeatMatchList)
# unMatchDf = dfam.getUnmatchInfo(DRrepeatMatchList)

matchCount:579	dfamCount:602	Ratio:0.9617940199335548


0.9617940199335548

In [22]:
# dfam.familyMatchRatio(RDmatchedFamilyAccList)
# dfam.matchRatio(RDrepeatMatchList)
# unMatchDf = dfam.getUnmatchInfo(RDrepeatMatchList)

In [23]:
# lengthScatterDistributionPlot(list(unMatchDf["length"]))

In [44]:
# non-identical
dfam2 = DfamEvaluation(repeatPositionList, hitFileName='chrX_LTR_dm6_dfam.nrph.hits')
repeatPositionLookupDic2 = dfam2.positionBucketClassifier()
dfamPositionList2 = dfam2.getDfamPositionList()
DRrepeatMatchList2, DRmatchedFamilyAccList2, DRmatchedFamilyNameList2 = dfam2.checkDfamMatchWithRepeat()
dfam.matchRatio(DRrepeatMatchList2)

matchCount:597	dfamCount:602	Ratio:0.9916943521594684


0.9916943521594684

In [46]:
textfile = open(f"../outputFile/txtFile/DRrepeatMatchList_nonIdenticalStatePosition.txt", "w")
for element in DRrepeatMatchList2:
    textfile.write(str(element) + "\n")
textfile.close()

In [24]:
def getSequenceLengthAnalsis(inputLengthList, num=10):
    """
    1. count of common length 
    2. statistic info
    3. distribution plot
    """
    mostCommonTable(Counter(inputLengthList).most_common(num), num)
    getStatisticData(inputLengthList)
    sortedCounterList = listToSortedCounter(inputLengthList)
    basicPlot(sortedCounterList)

In [25]:
# totalDfam = DfamEvaluation(repeatPositionList, hitFileName="chrX_dm6_dfam.nrph.hits")
# totalRepeatPositionLookupDic = totalDfam.positionBucketClassifier()
# totalDfamPositionList = totalDfam.getDfamPositionList()
# unionAndFilter_Position = pd.read_csv('../outputFile/txtFile/unionAndFilter_Position.csv')
# intersectionAndFilter_Position = pd.read_csv('../outputFile/txtFile/intersectionAndFilter_Position.csv')
# Filter_Position = pd.read_csv('../outputFile/txtFile/Filter_Position.csv')
# NonFilter_Position = pd.read_csv('../outputFile/txtFile/NonFilter_Position.csv')

# unionAndFilter_Counter = listToSortedCounter((unionAndFilter_Position['length']))
# intersectionAndFilter_Counter = listToSortedCounter((intersectionAndFilter_Position['length']))

In [26]:
# df = pd.DataFrame(columns=["x", "y", "type"], dtype=float)
# for row in unionAndFilter_Counter:
#     df = df.append({"x": row[0], "y": row[1], "type": "unionAndFilter"}, ignore_index=True)
# for row in intersectionAndFilter_Counter:
#     df = df.append({"x": row[0], "y": row[1], "type": "intersectionAndFilter"}, ignore_index=True)

# df.fillna(np.nan, inplace=True)
# fig, ax = plt.subplots(figsize=(10, 6), dpi=300)
# sns.set_style("whitegrid")
# sns.lineplot(data=df, x="x", y="y", hue="type", palette="Set1")
# ax.set_xlabel("Length", size=15)
# ax.set_ylabel("Count", size=15)
# ax.set_xlim(0, 1500)

In [27]:
# Dfam ref sequence 
# dfamSeqLenList = [ i.endIdx - i.startIdx for i in dfamPositionList ]
# getSequenceLengthAnalsis(dfamSeqLenList)

In [28]:
# Repeat sequence
# repeatFragmentLenList = [ i.endIdx - i.startIdx for i in repeatPositionList ]
# getSequenceLengthAnalsis(repeatFragmentLenList)

In [29]:
# def consensusSeqSimilarity(consensusSeq, seqDf):
#     print("hihi", len(seqDf))
#     seqSimilarityList = []
#     for targetSeq in seqDf:
#         alignments = pairwise2.align.globalxx(targetSeq, consensusSeq)
#         targetLength = len(targetSeq)
#         similarityPercentage = round(alignments[0].score / targetLength, 2)
#         seqSimilarityList.append(similarityPercentage)
#     return seqSimilarityList

# repeatMatchIdxList = []
# for idx, value in enumerate(RDrepeatMatchList):
#     if value == True:
#         repeatMatchIdxList.append(idx)
# repeatBasePositionList = [repeatPositionList[i] for i in repeatMatchIdxList]
# repeatSeqDf = pd.DataFrame(columns=['startIdx','endIdx', 'length', 'seq'])
# for i in repeatBasePositionList:
#     repeatSeqDf = repeatSeqDf.append({'startIdx':i.startIdx ,'endIdx': i.endIdx, 'length': (i.endIdx- i.startIdx), 'seq': str(parseFastaA[0][i.startIdx:i.endIdx])}, ignore_index=True)
# conParseFasta = parseFasta(
#     "DF0001696_ROO_LTR",
#     "./Evaluation/Source/DF0001696_ROO_LTR.fa",
#     "*",
#     matchMode=False,
# )
# consensusSeq = conParseFasta[0].upper()
# repeatDf = pd.read_csv('./Evaluation/Source/chrX_ROO_LTR_repeatSeq.csv')
# seqDf = repeatDf["seq"]
# seqSimilarityList = consensusSeqSimilarity(consensusSeq, seqDf)
# pd.Series(seqSimilarityList).describe()

In [30]:
# Test cutter A
repeatPositionListA = seqA.getRepeatPositionList()
dfamA = DfamEvaluation(repeatPositionListA)
repeatPositionLookupDicA = dfamA.positionBucketClassifier()
dfamPositionListA = dfamA.getDfamPositionList()
dfamPositionLookupDicA = dfamA.positionBucketClassifier()
DRrepeatMatchListA, DRmatchedFamilyAccListA, DRmatchedFamilyNameListA = dfamA.checkDfamMatchWithRepeat()

KeyboardInterrupt: 

In [ ]:
totalLen = len(DRrepeatMatchListA)
matchLenA = len(list(filter(lambda x: x, DRrepeatMatchListA)))
ratio = matchLenA / totalLen
print(f"matchCount:{matchLenA}\tdfamCount:{totalLen}\tRatio:{ratio}")

matchCount:1	dfamCount:602	Ratio:0.0016611295681063123


In [ ]:
# Test cutter B
repeatPositionListB = seqB.getRepeatPositionList()
dfamB = DfamEvaluation(repeatPositionListB)
repeatPositionLookupDicB = dfamB.positionBucketClassifier()
dfamPositionListB = dfamB.getDfamPositionList()
dfamPositionLookupDicB = dfamB.positionBucketClassifier()
DRrepeatMatchListB, DRmatchedFamilyAccListB, DRmatchedFamilyNameListB = dfamB.checkDfamMatchWithRepeat()

In [ ]:
totalLen = len(DRrepeatMatchListB)
matchLenB = len(list(filter(lambda x: x, DRrepeatMatchListB)))
ratio = matchLenB / totalLen
print(f"matchCount:{matchLenB}\tdfamCount:{totalLen}\tRatio:{ratio}")

matchCount:1	dfamCount:602	Ratio:0.0016611295681063123


In [ ]:
# total = 597
# middle = matchLenA+matchLenB - total
# plt.figure(linewidth=10, facecolor="white", dpi=1200)
# # plt.figure(linewidth=10, facecolor="white")
# v = venn2(subsets = (matchLenA-middle, matchLenB-middle, middle), set_labels = (f'CutterA - {cutterA} ', f'CutterB - {cutterB}'), set_colors=(colorA, colorB))
# plt.show()
